In [1]:
import pickle
import os
import torch
import numpy as np
import IPython.display as ipd
from util.evaluate import Evaluator
from factory.AutoVC import AutoVC
from factory.MetaConv import MetaConv
from factory.MetaPool import MetaPool
from factory.AutoVC_Adjust import AutoVC_Adjust

In [8]:
class Config:
    def __init__(self,root):
        self.root = root
        self.num_speaker = 40
        self.batch_size = 2
        self.erroment_uttr_idx = 16
        self.max_uttr_idx = 68
        self.len_crop = 176
        self.device = "cuda:0"
        self.judge = torch.load('model/static/metadv_vctk80.pt').to(self.device)
        self.all_speaker = sorted(next(iter(os.walk(root)))[1][:self.num_speaker])
        self.metadata = pickle.load(open(f'{root}/train.pkl', "rb"))[:self.num_speaker]

In [9]:
config = Config('train_data_vctk_ver2')

In [4]:

isAdjust = False
org = AutoVC(44,256,512,22).to(config.device)
org.load_state_dict(torch.load("model/autovc_en.pt", map_location=config.device))

pool = MetaPool(44,256,512,22).to(config.device)
pool.load_state_dict(torch.load("model/metapool_en.pt", map_location=config.device))

conv = MetaConv(44,256,512,22).to(config.device)
conv.load_state_dict(torch.load("model/metaconv_en.pt", map_location=config.device))


<All keys matched successfully>

In [5]:
#isAdjust = True
#org_ad = AutoVC_Adjust(44,256,512,22).to(config.device)
#org_ad.load_state_dict(torch.load("model/autovc_adjust_en.pt", map_location=config.device))

In [10]:
E = Evaluator(config)

Detect Judge ! generate all Real Data d-vector
Processing --- ID:0 Speaker:p225 ---


AttributeError: 'tuple' object has no attribute 'detach'

In [7]:
real_cos_result= E.get_real_data_cos()

Processing --- ID:0 Speaker:p225 ---
Processing --- ID:1 Speaker:p226 ---
Processing --- ID:2 Speaker:p227 ---
Processing --- ID:3 Speaker:p228 ---
Processing --- ID:4 Speaker:p229 ---


In [8]:
np.diagonal(real_cos_result).mean(), (np.sum(real_cos_result) - (np.sum(np.diagonal(real_cos_result)))) /(config.num_speaker*(config.num_speaker-1))

(0.9492713689804078, 0.7726693868637085)

In [9]:
org_cos_result = E.generate_result([org],isAdjust)

Now Processing --- p225
Reconstruct ---- p225 to p225
Trans --- p225 to p226
Trans --- p225 to p227
Trans --- p225 to p228
Trans --- p225 to p229
Now Processing --- p226
Trans --- p226 to p225
Reconstruct ---- p226 to p226
Trans --- p226 to p227
Trans --- p226 to p228
Trans --- p226 to p229
Now Processing --- p227
Trans --- p227 to p225
Trans --- p227 to p226
Reconstruct ---- p227 to p227
Trans --- p227 to p228
Trans --- p227 to p229
Now Processing --- p228
Trans --- p228 to p225
Trans --- p228 to p226
Trans --- p228 to p227
Reconstruct ---- p228 to p228
Trans --- p228 to p229
Now Processing --- p229
Trans --- p229 to p225
Trans --- p229 to p226
Trans --- p229 to p227
Trans --- p229 to p228
Reconstruct ---- p229 to p229


In [10]:
pool_cos_result = E.generate_result([pool],isAdjust)

Now Processing --- p225
Reconstruct ---- p225 to p225
Trans --- p225 to p226
Trans --- p225 to p227
Trans --- p225 to p228
Trans --- p225 to p229
Now Processing --- p226
Trans --- p226 to p225
Reconstruct ---- p226 to p226
Trans --- p226 to p227
Trans --- p226 to p228
Trans --- p226 to p229
Now Processing --- p227
Trans --- p227 to p225
Trans --- p227 to p226
Reconstruct ---- p227 to p227
Trans --- p227 to p228
Trans --- p227 to p229
Now Processing --- p228
Trans --- p228 to p225
Trans --- p228 to p226
Trans --- p228 to p227
Reconstruct ---- p228 to p228
Trans --- p228 to p229
Now Processing --- p229
Trans --- p229 to p225
Trans --- p229 to p226
Trans --- p229 to p227
Trans --- p229 to p228
Reconstruct ---- p229 to p229


In [11]:
conv_cos_result = E.generate_result([conv],isAdjust)

Now Processing --- p225
Reconstruct ---- p225 to p225
Trans --- p225 to p226
Trans --- p225 to p227
Trans --- p225 to p228
Trans --- p225 to p229
Now Processing --- p226
Trans --- p226 to p225
Reconstruct ---- p226 to p226
Trans --- p226 to p227
Trans --- p226 to p228
Trans --- p226 to p229
Now Processing --- p227
Trans --- p227 to p225
Trans --- p227 to p226
Reconstruct ---- p227 to p227
Trans --- p227 to p228
Trans --- p227 to p229
Now Processing --- p228
Trans --- p228 to p225
Trans --- p228 to p226
Trans --- p228 to p227
Reconstruct ---- p228 to p228
Trans --- p228 to p229
Now Processing --- p229
Trans --- p229 to p225
Trans --- p229 to p226
Trans --- p229 to p227
Trans --- p229 to p228
Reconstruct ---- p229 to p229


In [14]:
E.get_cos_rc(org_cos_result[0])[0],E.get_cos_rc(pool_cos_result[0])[0],E.get_cos_rc(conv_cos_result[0])[0]

(0.9516497850418091, 0.946908175945282, 0.939293360710144)

In [15]:
E.get_cos_trans(org_cos_result[0])[0],E.get_cos_trans(pool_cos_result[0])[0],E.get_cos_trans(conv_cos_result[0])[0]

(0.9536625027656556, 0.9296765953302384, 0.9245863527059555)

In [ ]:
all_rc_result = []
all_o_rc_result = []
all_trans_result = []
all_o_trans_result = []
for _ in range(4):
    all_cos_result = E.generate_result([org],isAdjust)
    rc,o_rc = E.get_cos_rc(all_cos_result[0])
    trans,o_trans = E.get_cos_trans(all_cos_result[0])
    all_rc_result.append(rc)
    all_o_rc_result.append(o_rc)
    all_trans_result.append(trans)
    all_o_trans_result.append(o_trans)

In [ ]:
np.array(all_rc_result).mean(),np.array(all_trans_result).mean()

In [ ]:
np.array(all_o_rc_result).mean(),np.array(all_o_trans_result).mean()

In [ ]:
np.array(all_rc_result).std(),np.array(all_trans_result).std()

In [ ]:
np.array(all_o_rc_result).std(),np.array(all_o_trans_result).std()